# HOW MANY CELLS ?

In this notebook we will count the number of cells on each image of the cleaned database.

In [24]:
# We import the useful functions from the script save in the src folder

from src.utils import img_greying
from src.utils import img_blurring
from src.utils import cell_maskification
from src.utils import smooth
from src.utils import small_hole_removing
from src.utils import small_object_removing
from src.utils import cell_erosion
from src.utils import cell_labelisation
from src.utils import cell_count
from src.utils import dataframe_cells_centroid_creation

# We import some existent libraries 

import pandas as pd
from tifffile import imread
import napari
from skimage import restoration
import skimage.filters as filters
from skimage.morphology import disk
from skimage import morphology
from skimage.feature import peak_local_max
from skimage.morphology import convex_hull_image
from skimage.segmentation import watershed
from scipy import ndimage as ndi
import numpy as np
import pandas as pd
from skimage import measure
import matplotlib.pyplot as plt
import tifffile
from scipy import ndimage as ndi
import skimage
from skimage.color import rgb2gray
from typing import List
from magicgui import magicgui
from napari.types import ImageData, LabelsData, LayerDataTuple
import numpy as np
import glob
from skimage import feature

# ALL USER CHOICE

In [25]:
data_folder_path = "/home/criuser/Bureau/Informatique/Projet"

# ANSWER

## Preliminary steps

In [26]:
# We import our csv containing the cell database cleaned ()

path = data_folder_path  + '/data/cleaned/global_database_cleaned.csv'                                                                                         
df = pd.read_csv(path)                                                                     
df

,plant_name,cell_fname,leaf_id,leaf_orientation,group
0,Ceratostigma_minus,3-07-400x-1-01,3.70,abaxial,1
1,Ceratostigma_minus,3-07-400x-1-02,3.70,abaxial,1
2,Ceratostigma_minus,3-07-400x-1-03,3.70,abaxial,1
3,Ceratostigma_minus,3-07-400x-1-04,3.70,abaxial,1
4,Ceratostigma_minus,3-07-400x-2-01,3.70,adaxial,1
...,...,...,...,...,...
84,Hemerocallis_fulva,7-24-400x-1-05,7.24,abaxial,5
85,Jasminum fruticans,8-06-700x-2-01,8.60,NaN,3
86,Jasminum fruticans,8-06-700x-2-02,8.60,NaN,3
87,Jasminum fruticans,8-06-700x-2-03,8.60,NaN,3


## Counting and centroids determination

We create a function able to count the number of cell of an image from this image (and some parameters) and return this number and the label corresponding to each cell. It is enought flexible to be use on new images just by quickly changing the parameters values.

In [27]:
def cell_count_and_label_from_img(
    img_original, 
    blurring_intensity, 
    local_threshold_block_size, 
    closing_radius, 
    opening_radius, 
    min_size_hole,
    min_size_object,
    erosion_radius
    ):

# Preprocessing 
    img_grey = img_greying(img_original)
    img_blurred = img_blurring(img_grey, blurring_intensity)

# Processing 
    mask_cell = cell_maskification(img_blurred, local_threshold_block_size)
    mask_smooth = smooth(mask_cell, closing_radius, opening_radius)
    mask_no_small_hole = small_hole_removing(mask_smooth, min_size_hole)
    mask_no_small_object = small_object_removing(mask_no_small_hole, min_size_object)
    mask_erroded = cell_erosion(mask_no_small_object, erosion_radius)
    mask_no_small_object_2 = small_object_removing(mask_erroded, min_size_object)

# Postprocessing 
    mask_cell_label = cell_labelisation(mask_no_small_object_2)
    cell_number = cell_count(mask_cell_label)
    return cell_number, mask_cell_label

## Group 1 images

In [35]:
# Group 1 images parameters 

blurring_intensity = 10
local_threshold_block_size = 15 
closing_radius = 2
opening_radius = 1
min_size_object = 2000
min_size_hole = 2000
erosion_radius = 3

In [36]:
# Extraction of the group 1 images 

df_group_1 = df.loc[df['group'] == 1]
df_how_many_cells_1 = pd.DataFrame(columns = ['cell_fname','nb_cells', 'centroids_fname'])

for cell_fname in df_group_1['cell_fname'] :
    img_name = cell_fname +'.tif'
    img_path = data_folder_path + "/data/cleaned/cell_database_cleaned/" + img_name 
    img_original = imread(img_path)
    

# Number of cells for the group 1 images

    cell_number = cell_count_and_label_from_img(
                        img_original, 
                        blurring_intensity, 
                        local_threshold_block_size, 
                        closing_radius, 
                        opening_radius, 
                        min_size_hole,
                        min_size_object,
                        erosion_radius
                        )[0]      # [0] because we want the cell number with is the first variable returned          

# Centroid determination

    mask_cell_label = cell_count_and_label_from_img(
                        img_original, 
                        blurring_intensity, 
                        local_threshold_block_size, 
                        closing_radius, 
                        opening_radius, 
                        min_size_hole,
                        min_size_object,
                        erosion_radius
                        )[1]   # [1] because we want the label corresponding to each cell with is the 2nd variable returned 
    df_cells_centroid = dataframe_cells_centroid_creation(mask_cell_label)

# cells_centroid-image_name.csv creation and saving

    df_cells_centroid.to_csv(data_folder_path + "/results/cells_centroid_csv" + "/" + "cells_centroid_" + cell_fname + ".csv")
    
# how_many_cells dataframe filing for the group 1 images

    df_intermediate = pd.DataFrame({"cell_fname":[cell_fname], "nb_cells":[cell_number], "centroids_fname":["cells_centroid_" + cell_fname + ".csv"]})
    df_how_many_cells_1 = pd.concat([df_how_many_cells_1, df_intermediate], ignore_index=True)

## Group 2 images

In [37]:
# Group 2 images parameters 

blurring_intensity = 10
local_threshold_block_size = 35 
closing_radius = 2
opening_radius = 1
min_size_object = 2000
min_size_hole = 2000
erosion_radius = 3

In [38]:
# Extraction of the group 2 images 

df_group_2 = df.loc[df['group'] == 2]
df_how_many_cells_2 = pd.DataFrame(columns = ['cell_fname','nb_cells', 'centroids_fname'])

for cell_fname in df_group_2['cell_fname'] :
    img_name = cell_fname +'.tif'
    img_path = data_folder_path + "/data/cleaned/cell_database_cleaned/" + img_name 
    img_original = imread(img_path)
    

# Number of cells for the group 2 images

    cell_number = cell_count_and_label_from_img(
                        img_original, 
                        blurring_intensity, 
                        local_threshold_block_size, 
                        closing_radius, 
                        opening_radius, 
                        min_size_hole,
                        min_size_object,
                        erosion_radius
                        )[0]      # [0] because we want the cell number with is the first variable returned          

# Centroid determination

    mask_cell_label = cell_count_and_label_from_img(
                        img_original, 
                        blurring_intensity, 
                        local_threshold_block_size, 
                        closing_radius, 
                        opening_radius, 
                        min_size_hole,
                        min_size_object,
                        erosion_radius
                        )[1]   # [1] because we want the label corresponding to each cell with is the 2nd variable returned 
    df_cells_centroid = dataframe_cells_centroid_creation(mask_cell_label)

# cells_centroid-image_name.csv creation and saving

    df_cells_centroid.to_csv(data_folder_path + "/results/cells_centroid_csv" + "/" + "cells_centroid_" + cell_fname + ".csv")
    
# how_many_cells dataframe filing for the group 1 images

    df_intermediate = pd.DataFrame({"cell_fname":[cell_fname], "nb_cells":[cell_number], "centroids_fname":["cells_centroid_" + cell_fname + ".csv"]})
    df_how_many_cells_2 = pd.concat([df_how_many_cells_2, df_intermediate], ignore_index=True)

## Group 3 images 

In [39]:
# Group 3 images parameters 

blurring_intensity = 10
local_threshold_block_size = 15 
closing_radius = 2
opening_radius = 1
min_size_object = 2000
min_size_hole = 2000
erosion_radius = 3

In [40]:
# Extraction of the group 3 images 

df_group_3= df.loc[df['group'] == 3]
df_how_many_cells_3 = pd.DataFrame(columns = ['cell_fname','nb_cells', 'centroids_fname'])

for cell_fname in df_group_3['cell_fname'] :
    img_name = cell_fname +'.tif'
    img_path = data_folder_path + "/data/cleaned/cell_database_cleaned/" + img_name 
    img_original = imread(img_path)
    

# Number of cells for the group 3 images

    cell_number = cell_count_and_label_from_img(
                        img_original, 
                        blurring_intensity, 
                        local_threshold_block_size, 
                        closing_radius, 
                        opening_radius, 
                        min_size_hole,
                        min_size_object,
                        erosion_radius
                        )[0]      # [0] because we want the cell number with is the first variable returned          

# Centroid determination

    mask_cell_label = cell_count_and_label_from_img(
                        img_original, 
                        blurring_intensity, 
                        local_threshold_block_size, 
                        closing_radius, 
                        opening_radius, 
                        min_size_hole,
                        min_size_object,
                        erosion_radius
                        )[1]   # [1] because we want the label corresponding to each cell with is the 2nd variable returned 
    df_cells_centroid = dataframe_cells_centroid_creation(mask_cell_label)

# cells_centroid-image_name.csv creation and saving

    df_cells_centroid.to_csv(data_folder_path + "/results/cells_centroid_csv" + "/" + "cells_centroid_" + cell_fname + ".csv")
    
# how_many_cells dataframe filing for the group 1 images

    df_intermediate = pd.DataFrame({"cell_fname":[cell_fname], "nb_cells":[cell_number], "centroids_fname":["cells_centroid_" + cell_fname + ".csv"]})
    df_how_many_cells_3 = pd.concat([df_how_many_cells_3, df_intermediate], ignore_index=True)

## Group 4 images

In [42]:
# Group 4 images parameters 

blurring_intensity = 10
local_threshold_block_size = 35 
closing_radius = 2
opening_radius = 1
min_size_object = 2000
min_size_hole = 3000
erosion_radius = 3

In [43]:
# Extraction of the group 4 images 

df_group_4 = df.loc[df['group'] == 4]
df_how_many_cells_4 = pd.DataFrame(columns = ['cell_fname','nb_cells', 'centroids_fname'])

for cell_fname in df_group_4['cell_fname'] :
    img_name = cell_fname +'.tif'
    img_path = data_folder_path + "/data/cleaned/cell_database_cleaned/" + img_name 
    img_original = imread(img_path)
    

# Number of cells for the group 4 images

    cell_number = cell_count_and_label_from_img(
                        img_original, 
                        blurring_intensity, 
                        local_threshold_block_size, 
                        closing_radius, 
                        opening_radius, 
                        min_size_hole,
                        min_size_object,
                        erosion_radius
                        )[0]      # [0] because we want the cell number with is the first variable returned          

# Centroid determination

    mask_cell_label = cell_count_and_label_from_img(
                        img_original, 
                        blurring_intensity, 
                        local_threshold_block_size, 
                        closing_radius, 
                        opening_radius, 
                        min_size_hole,
                        min_size_object,
                        erosion_radius
                        )[1]   # [1] because we want the label corresponding to each cell with is the 2nd variable returned 
    df_cells_centroid = dataframe_cells_centroid_creation(mask_cell_label)

# cells_centroid-image_name.csv creation and saving

    df_cells_centroid.to_csv(data_folder_path + "/results/cells_centroid_csv" + "/" + "cells_centroid_" + cell_fname + ".csv")
    
# how_many_cells dataframe filing for the group 1 images

    df_intermediate = pd.DataFrame({"cell_fname":[cell_fname], "nb_cells":[cell_number], "centroids_fname":["cells_centroid_" + cell_fname + ".csv"]})
    df_how_many_cells_4 = pd.concat([df_how_many_cells_4, df_intermediate], ignore_index=True)

## Group 5 images

In [45]:
# Group 5 images parameters 

blurring_intensity = 10
local_threshold_block_size = 15 
closing_radius = 2
opening_radius = 1
min_size_object = 2000
min_size_hole = 2000
erosion_radius = 3

In [46]:
# Extraction of the group 5 images 

df_group_5 = df.loc[df['group'] == 5]
df_how_many_cells_5 = pd.DataFrame(columns = ['cell_fname','nb_cells', 'centroids_fname'])

for cell_fname in df_group_5['cell_fname'] :
    img_name = cell_fname +'.tif'
    img_path = data_folder_path + "/data/cleaned/cell_database_cleaned/" + img_name 
    img_original = imread(img_path)
    

# Number of cells for the group 5 images

    cell_number = cell_count_and_label_from_img(
                        img_original, 
                        blurring_intensity, 
                        local_threshold_block_size, 
                        closing_radius, 
                        opening_radius, 
                        min_size_hole,
                        min_size_object,
                        erosion_radius
                        )[0]      # [0] because we want the cell number with is the first variable returned          

# Centroid determination

    mask_cell_label = cell_count_and_label_from_img(
                        img_original, 
                        blurring_intensity, 
                        local_threshold_block_size, 
                        closing_radius, 
                        opening_radius, 
                        min_size_hole,
                        min_size_object,
                        erosion_radius
                        )[1]   # [1] because we want the label corresponding to each cell with is the 2nd variable returned 
    df_cells_centroid = dataframe_cells_centroid_creation(mask_cell_label)

# cells_centroid-image_name.csv creation and saving

    df_cells_centroid.to_csv(data_folder_path + "/results/cells_centroid_csv" + "/" + "cells_centroid_" + cell_fname + ".csv")
    
# how_many_cells dataframe filing for the group 1 images

    df_intermediate = pd.DataFrame({"cell_fname":[cell_fname], "nb_cells":[cell_number], "centroids_fname":["cells_centroid_" + cell_fname + ".csv"]})
    df_how_many_cells_5 = pd.concat([df_how_many_cells_5, df_intermediate], ignore_index=True)

## How_many_cells.csv final creation and saving

In [48]:
df_how_many_cells = pd.concat([df_how_many_cells_1, 
                               df_how_many_cells_2, 
                               df_how_many_cells_3, 
                               df_how_many_cells_4,
                               df_how_many_cells_5], ignore_index=True)

df_how_many_cells.to_csv(data_folder_path + "/results/" + "how_many_cells.csv")